# Evaluation

In [1]:
utils_path = "../utils/"
corpus_path = "../datasets/cantemist_v6/"
sub_task_path = "cantemist-norm/"
test_gs_path = corpus_path + "test-set/" + sub_task_path

In [2]:
import tensorflow as tf

# Auxiliary components
import sys
sys.path.insert(0, utils_path)
from nlp_utils import *

RES_DIR = "../results/Cantemist/final_exec/"

# GS data
df_test_gs = format_ner_gs(test_gs_path, subtask='norm')

2022-09-14 09:36:49.989280: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


## Evaluation

In [3]:
def check_ner_norm_performance(model_name, arr_execs):
    """
    Sanity-check procedure that prints the NORM performance of each single model execution.
    """
    for i_exec in arr_execs:
        print("Exec " + str(i_exec) + ":")
        df_test_preds = pd.read_csv(RES_DIR + "df_test_preds_c_hier_task_cls_" + \
                str(model_name) + "_" + str(i_exec) + ".csv", header=0, sep='\t')
        print("NORM performance:", calculate_ner_metrics(
            gs=df_test_gs, 
            pred=format_ner_pred_df(
                gs_path=test_gs_path, 
                df_preds=df_test_preds, 
                subtask='norm'
            ),
            subtask='norm'
        ), end="\n\n")

In [4]:
def model_performance(dict_names_execs, 
                      df_gs=df_test_gs, path_gs=test_gs_path,
                      round_n=3, multi_task=False):
    """
    Generate a pd.DataFrame with the statistics of the performance of each model.
    
    dict_names_execs: each key is a string with the model name, and 
                      each value is a list with the execs of the corresponding model.
    """
    res_dict = {}
    for model_name in dict_names_execs:
        p_res, r_res, f1_res = [], [], []
        for i_exec in dict_names_execs[model_name]:
            if multi_task:
                df_test_preds = pd.read_csv(RES_DIR + "df_test_preds_multi_task_ner_" + str(i_exec) + "_c_hier_task_cls_" + \
                        str(model_name) + "_" + str(i_exec) + ".csv", header=0, sep='\t')
            else:
                df_test_preds = pd.read_csv(RES_DIR + "df_test_preds_c_hier_task_cls_" + \
                        str(model_name) + "_" + str(i_exec) + ".csv", header=0, sep='\t')
            p, r, f1 = calculate_ner_metrics(
                gs=df_gs, 
                pred=format_ner_pred_df(
                    gs_path=path_gs, 
                    df_preds=df_test_preds, 
                    subtask='norm'
                ),
                subtask='norm'
            )
            p_res.append(p)
            r_res.append(r)
            f1_res.append(f1)
        p_res_stat = pd.Series(p_res).describe()
        r_res_stat = pd.Series(r_res).describe()
        f1_res_stat = pd.Series(f1_res).describe()
        res_dict[model_name] = {"P_avg": round(p_res_stat['mean'], round_n), "P_std": round(p_res_stat['std'], round_n), 
                                "P_max": round(p_res_stat['max'], round_n),
                                "R_avg": round(r_res_stat['mean'], round_n), "R_std": round(r_res_stat['std'], round_n), 
                                "R_max": round(r_res_stat['max'], round_n),
                                "F1_avg": round(f1_res_stat['mean'], round_n), "F1_std": round(f1_res_stat['std'], round_n), 
                                "F1_max": round(f1_res_stat['max'], round_n)}
    return pd.DataFrame(res_dict, index=["P_avg", "P_std", "P_max", 
                                         "R_avg", "R_std", "R_max", 
                                         "F1_avg", "F1_std", "F1_max"]).transpose()    

In [5]:
def format_df_paper(df_res):
    arr_metrics = ["P", "R", "F1"]
    arr_cols = []
    for metric in arr_metrics:
        df_res[metric + '_avg_std'] = df_res.apply(
            lambda x: "." + str(x[metric + '_avg']).split('.')[-1] + " ± " + \
                "." + str(x[metric + '_std']).split('.')[-1], 
            axis=1
        )
        df_res[metric + '_max'] = df_res[metric + '_max'].apply(
            lambda x: "." + str(x).split('.')[-1]
        )
        arr_cols += [metric + '_avg_std', metric + '_max']
    return df_res[arr_cols]

In [5]:
# Sanity check

In [6]:
m_name = "xlmr"
execs = [1, 2, 3, 4, 5]

check_ner_norm_performance(model_name=m_name, arr_execs=execs)

Exec 1:
NORM performance: (0.8215, 0.8219, 0.8217)

Exec 2:
NORM performance: (0.8131, 0.8203, 0.8167)

Exec 3:
NORM performance: (0.816, 0.8128, 0.8144)

Exec 4:
NORM performance: (0.8075, 0.8153, 0.8114)

Exec 5:
NORM performance: (0.8112, 0.815, 0.8131)



In [7]:
m_name = "xlmr_galen"
execs = [1, 2, 3, 4, 5]

check_ner_norm_performance(model_name=m_name, arr_execs=execs)

Exec 1:
NORM performance: (0.8255, 0.8216, 0.8236)

Exec 2:
NORM performance: (0.8274, 0.8103, 0.8188)

Exec 3:
NORM performance: (0.8325, 0.8318, 0.8322)

Exec 4:
NORM performance: (0.8119, 0.8282, 0.82)

Exec 5:
NORM performance: (0.8219, 0.8192, 0.8205)



## Paper

In [10]:
model_performance(
    {
        'beto': [1, 2, 3, 4, 5], 
        'beto_galen': [1, 2, 3, 4, 5],
        'mbert': [1, 2, 3, 4, 5], 
        'mbert_galen': [1, 2, 3, 4, 5],
        'xlmr': [1, 2, 3, 4, 5], 
        'xlmr_galen': [1, 2, 3, 4, 5]
    }
)

,P_avg,P_std,P_max,R_avg,R_std,R_max,F1_avg,F1_std,F1_max
beto,0.818,0.007,0.824,0.811,0.004,0.816,0.814,0.003,0.819
beto_galen,0.825,0.005,0.833,0.821,0.005,0.826,0.823,0.004,0.829
mbert,0.819,0.009,0.832,0.818,0.009,0.830,0.818,0.002,0.820
mbert_galen,0.828,0.007,0.839,0.825,0.004,0.830,0.826,0.004,0.832
xlmr,0.814,0.005,0.822,0.817,0.004,0.822,0.815,0.004,0.822
xlmr_galen,0.824,0.008,0.832,0.822,0.008,0.832,0.823,0.005,0.832


In [11]:
format_df_paper(
    model_performance(
        {
            'beto': [1, 2, 3, 4, 5], 
            'beto_galen': [1, 2, 3, 4, 5],
            'mbert': [1, 2, 3, 4, 5], 
            'mbert_galen': [1, 2, 3, 4, 5],
            'xlmr': [1, 2, 3, 4, 5], 
            'xlmr_galen': [1, 2, 3, 4, 5]
        }
    )
)

,P_avg_std,P_max,R_avg_std,R_max,F1_avg_std,F1_max
beto,.818 ± .007,.824,.811 ± .004,.816,.814 ± .003,.819
beto_galen,.825 ± .005,.833,.821 ± .005,.826,.823 ± .004,.829
mbert,.819 ± .009,.832,.818 ± .009,.83,.818 ± .002,.82
mbert_galen,.828 ± .007,.839,.825 ± .004,.83,.826 ± .004,.832
xlmr,.814 ± .005,.822,.817 ± .004,.822,.815 ± .004,.822
xlmr_galen,.824 ± .008,.832,.822 ± .008,.832,.823 ± .005,.832


Save the (F1) performance of all executions of all models

In [6]:
def model_f1_values(dict_names_execs, 
                    df_gs=df_test_gs, path_gs=test_gs_path):
    """
    Generate a vector containing the F1 performance of all executions of all models, in the given order.
    
    dict_names_execs: each key is a string with the model name, and 
                      each value is a list with the random execs of the corresponding model.
    """
    arr_values = []
    for model_name in dict_names_execs:
        for i_exec in dict_names_execs[model_name]:
            df_test_preds = pd.read_csv(RES_DIR + "df_test_preds_c_hier_task_cls_" + \
                    str(model_name) + "_" + str(i_exec) + ".csv", header=0, sep='\t')
            _, _, f1 = calculate_ner_metrics(
                gs=df_gs, 
                pred=format_ner_pred_df(
                    gs_path=path_gs, 
                    df_preds=df_test_preds, 
                    subtask='norm'
                ),
                subtask='norm'
            )
            arr_values.append(f1)
    return arr_values

In [ ]:
# NORM

In [7]:
arr_val = model_f1_values(
    {
        'beto': [1, 2, 3, 4, 5], 
        'beto_galen': [1, 2, 3, 4, 5],
        'mbert': [1, 2, 3, 4, 5], 
        'mbert_galen': [1, 2, 3, 4, 5],
        'xlmr': [1, 2, 3, 4, 5], 
        'xlmr_galen': [1, 2, 3, 4, 5]
    }
)

In [8]:
pd.DataFrame(arr_val).to_csv(RES_DIR + "norm_f1_exec_c_hier_task.csv", index=False, header=False, sep = '\t')

## Ensemble

### Load data

In [12]:
train_path = corpus_path + "train-set/" + sub_task_path
train_files = [f for f in os.listdir(train_path) if os.path.isfile(train_path + f) and f.split('.')[-1] == "txt"]
n_train_files = len(train_files)
train_data = load_text_files(train_files, train_path)
dev1_path = corpus_path + "dev-set1/" + sub_task_path
train_files.extend([f for f in os.listdir(dev1_path) if os.path.isfile(dev1_path + f) and f.split('.')[-1] == "txt"])
train_data.extend(load_text_files(train_files[n_train_files:], dev1_path))
df_text_train = pd.DataFrame({'doc_id': [s.split('.txt')[0] for s in train_files], 'raw_text': train_data})

In [13]:
dev_path = corpus_path + "dev-set2/" + sub_task_path
dev_files = [f for f in os.listdir(dev_path) if os.path.isfile(dev_path + f) and f.split('.')[-1] == "txt"]
dev_data = load_text_files(dev_files, dev_path)
df_text_dev = pd.DataFrame({'doc_id': [s.split('.txt')[0] for s in dev_files], 'raw_text': dev_data})

In [14]:
test_path = corpus_path + "test-set/" + sub_task_path
test_files = [f for f in os.listdir(test_path) if os.path.isfile(test_path + f) and f.split('.')[-1] == 'txt']
test_data = load_text_files(test_files, test_path)
df_text_test = pd.DataFrame({'doc_id': [s.split('.txt')[0] for s in test_files], 'raw_text': test_data})

In [15]:
train_ann_files = [train_path + f for f in os.listdir(train_path) if f.split('.')[-1] == "ann"]
train_ann_files.extend([dev1_path + f for f in os.listdir(dev1_path) if f.split('.')[-1] == "ann"])

In [16]:
df_codes_train_ner = process_brat_norm(train_ann_files).sort_values(["doc_id", "start", "end"])

In [17]:
df_codes_train_ner["code_pre"] = df_codes_train_ner["code"].apply(lambda x: x.split('/')[0])
df_codes_train_ner["code_suf"] = df_codes_train_ner["code"].apply(lambda x: '/'.join(x.split('/')[1:]))

In [18]:
assert ~df_codes_train_ner[["doc_id", "start", "end"]].duplicated().any()

In [19]:
df_codes_train_ner_final = df_codes_train_ner.copy()

In [20]:
print(df_codes_train_ner_final.shape[0])

9737


In [21]:
dev_ann_files = [dev_path + f for f in os.listdir(dev_path) if f.split('.')[-1] == "ann"]

In [22]:
df_codes_dev_ner = process_brat_norm(dev_ann_files).sort_values(["doc_id", "start", "end"])

In [23]:
df_codes_dev_ner["code_pre"] = df_codes_dev_ner["code"].apply(lambda x: x.split('/')[0])
df_codes_dev_ner["code_suf"] = df_codes_dev_ner["code"].apply(lambda x: '/'.join(x.split('/')[1:]))

In [24]:
assert ~df_codes_dev_ner[["doc_id", "start", "end"]].duplicated().any()

In [25]:
df_codes_dev_ner_final = df_codes_dev_ner.copy()

In [26]:
print(df_codes_dev_ner_final.shape[0])

2660


In [27]:
train_dev_codes_pre = sorted(set(df_codes_dev_ner_final["code_pre"].values).union(set(
    df_codes_train_ner_final["code_pre"].values
)))

In [28]:
len(train_dev_codes_pre)

307

In [29]:
train_dev_codes_suf = sorted(set(df_codes_dev_ner_final["code_suf"].values).union(set(df_codes_train_ner_final["code_suf"].values))) 

In [30]:
print(len(train_dev_codes_suf))

35


In [31]:
# Code-pre
code_pre_lab_encoder = {}
code_pre_lab_decoder = {}
i = 0
for code in train_dev_codes_pre:
    code_pre_lab_encoder[code] = i
    code_pre_lab_decoder[i] = code
    i += 1
    
code_pre_lab_encoder["O"] = i
code_pre_lab_decoder[i] = "O"

# Code-suf
code_suf_lab_encoder = {}
code_suf_lab_decoder = {}
i = 0
for code in train_dev_codes_suf:
    code_suf_lab_encoder[code] = i
    code_suf_lab_decoder[i] = code
    i += 1
    
code_suf_lab_encoder["O"] = i
code_suf_lab_decoder[i] = "O"

In [32]:
print(len(code_pre_lab_encoder), len(code_pre_lab_decoder))

308 308


In [35]:
print(len(code_suf_lab_encoder), len(code_suf_lab_decoder))

36 36


### Evaluation

In [36]:
ENS_EVAL_STRAT = 'sum'
RES_DIR_ENS = RES_DIR + "ensemble/"

subtask = 'norm'
subtask_ann = subtask + '-iob_code_suf'

arr_exec = [1, 2, 3, 4, 5]

In [37]:
def ens_performance(ens_name, ner_model_name, arr_model_name, 
                    res_dir=RES_DIR_ENS, prefix_name='', arr_exec=arr_exec, 
                    df_gs=df_test_gs, path_gs=test_gs_path, 
                    code_pre_lab_decoder=code_pre_lab_decoder, 
                    code_suf_lab_decoder=code_suf_lab_decoder, 
                    ens_eval_strategy=ENS_EVAL_STRAT, 
                    subtask=subtask, subtask_ann=subtask_ann):
    df_ens_ner = pd.read_csv(res_dir + "df_test_preds_ens_ner_" + ens_name + "_c_hier_task_cls_" + \
                             ner_model_name + "_ann.csv", header=0, sep='\t')
    ens_preds_code_pre, ens_preds_code_suf = [], []
    for model_name in arr_model_name:
        for i_exec in arr_exec:
            ens_preds_code_pre.append(np.load(file=res_dir + prefix_name + "test_preds_code_pre_ens_ner_" + ens_name + \
                                              "_c_hier_task_cls_" + model_name + "_" + str(i_exec) + ".npy"))
            ens_preds_code_suf.append(np.load(file=res_dir + prefix_name + "test_preds_code_suf_ens_ner_" + ens_name + \
                                              "_c_hier_task_cls_" + model_name + "_" + str(i_exec) + ".npy"))

    # Sanity check: all preds array have the same shape
    assert len(ens_preds_code_pre) == len(ens_preds_code_suf)
    for i in range(len(ens_preds_code_pre) - 1):
        assert ens_preds_code_pre[i].shape == ens_preds_code_pre[i + 1].shape
        assert ens_preds_code_suf[i].shape == ens_preds_code_suf[i + 1].shape
    
    print("Nº executions:", len(ens_preds_code_pre))

    if ens_eval_strategy == 'sum':
        ens_code_pre = np.sum(ens_preds_code_pre, axis=0)
        ens_code_suf = np.sum(ens_preds_code_suf, axis=0)
    else: # default 'prod'
        ens_code_pre = np.prod(ens_preds_code_pre, axis=0)
        ens_code_suf = np.prod(ens_preds_code_suf, axis=0)

    df_ens_preds = cls_code_norm_preds_brat_format(
        y_pred_cls=[ens_code_pre, ens_code_suf], 
        df_pred_ner=df_ens_ner, 
        code_decoder_list=[code_pre_lab_decoder, code_suf_lab_decoder],
        subtask=subtask_ann,
        codes_pre_o_mask=None,
        codes_pre_suf_mask=None
    )

    return df_ens_preds, calculate_ner_metrics(
        gs=df_gs, 
        pred=format_ner_pred_df(
            gs_path=path_gs, df_preds=df_ens_preds, 
            subtask=subtask
        ),
        subtask=subtask
    )

In [38]:
dic_ens_res = {}

### BETO

In [39]:
ens_name = 'beto'
ner_model_name = ens_name
arr_model_name = [ens_name]

In [40]:
df_ens_ann, res_metrics = ens_performance(
    ens_name=ens_name, 
    ner_model_name=ner_model_name, 
    arr_model_name=arr_model_name
)
print(res_metrics)

Nº executions: 5
(0.8358, 0.8282, 0.832)


In [41]:
dic_ens_res[ens_name] = res_metrics

In [40]:
df_ens_ann.to_csv(RES_DIR_ENS + "df_test_preds_c_hier_task_cls_" + '_'.join(arr_model_name) + \
                  ".csv", index=False, header=True, sep = '\t')

### BETO-Galén

In [42]:
ens_name = 'beto_galen'
ner_model_name = 'beto_galen'
arr_model_name = [ens_name]

In [43]:
df_ens_ann, res_metrics = ens_performance(
    ens_name=ens_name, 
    ner_model_name=ner_model_name, 
    arr_model_name=arr_model_name
)
print(res_metrics)

Nº executions: 5
(0.8384, 0.8337, 0.836)


In [44]:
dic_ens_res[ens_name] = res_metrics

In [43]:
df_ens_ann.to_csv(RES_DIR_ENS + "df_test_preds_c_hier_task_cls_" + '_'.join(arr_model_name) + \
                  ".csv", index=False, header=True, sep = '\t')

### mBERT

In [45]:
ens_name = 'mbert'
ner_model_name = 'mbert'
arr_model_name = [ens_name]

In [46]:
df_ens_ann, res_metrics = ens_performance(
    ens_name=ens_name, 
    ner_model_name=ner_model_name, 
    arr_model_name=arr_model_name
)
print(res_metrics)

Nº executions: 5
(0.8396, 0.8354, 0.8375)


In [47]:
dic_ens_res[ens_name] = res_metrics

In [46]:
df_ens_ann.to_csv(RES_DIR_ENS + "df_test_preds_c_hier_task_cls_" + '_'.join(arr_model_name) + \
                  ".csv", index=False, header=True, sep = '\t')

### mBERT-Galén

In [48]:
ens_name = 'mbert_galen'
ner_model_name = 'mbert_galen'
arr_model_name = [ens_name]

In [49]:
df_ens_ann, res_metrics = ens_performance(
    ens_name=ens_name, 
    ner_model_name=ner_model_name, 
    arr_model_name=arr_model_name
)
print(res_metrics)

Nº executions: 5
(0.8433, 0.8398, 0.8415)


In [50]:
dic_ens_res[ens_name] = res_metrics

In [49]:
df_ens_ann.to_csv(RES_DIR_ENS + "df_test_preds_c_hier_task_cls_" + '_'.join(arr_model_name) + \
                  ".csv", index=False, header=True, sep = '\t')

### XLM-R

In [51]:
ens_name = 'xlmr'
ner_model_name = 'xlmr'
arr_model_name = [ens_name]

In [52]:
df_ens_ann, res_metrics = ens_performance(
    ens_name=ens_name, 
    ner_model_name=ner_model_name, 
    arr_model_name=arr_model_name
)
print(res_metrics)

Nº executions: 5
(0.8348, 0.8332, 0.834)


In [53]:
dic_ens_res[ens_name] = res_metrics

In [52]:
df_ens_ann.to_csv(RES_DIR_ENS + "df_test_preds_c_hier_task_cls_" + '_'.join(arr_model_name) + \
                  ".csv", index=False, header=True, sep = '\t')

### XLM-R-Galén

In [54]:
ens_name = 'xlmr_galen'
ner_model_name = 'xlmr_galen'
arr_model_name = [ens_name]

In [55]:
df_ens_ann, res_metrics = ens_performance(
    ens_name=ens_name, 
    ner_model_name=ner_model_name, 
    arr_model_name=arr_model_name
)
print(res_metrics)

Nº executions: 5
(0.8426, 0.8382, 0.8403)


In [56]:
dic_ens_res[ens_name] = res_metrics

In [55]:
df_ens_ann.to_csv(RES_DIR_ENS + "df_test_preds_c_hier_task_cls_" + '_'.join(arr_model_name) + \
                  ".csv", index=False, header=True, sep = '\t')

### BETO + BETO-Galén

In [57]:
arr_model_name = ['beto', 'beto_galen']
ens_name = '_'.join(arr_model_name)
ner_model_name = 'beto'

In [58]:
df_ens_ann, res_metrics = ens_performance(
    ens_name=ens_name, 
    ner_model_name=ner_model_name, 
    arr_model_name=arr_model_name
)
print(res_metrics)

Nº executions: 10
(0.8453, 0.8346, 0.8399)


In [59]:
dic_ens_res[ens_name] = res_metrics

In [58]:
df_ens_ann.to_csv(RES_DIR_ENS + "df_test_preds_c_hier_task_cls_" + '_'.join(arr_model_name) + \
                  ".csv", index=False, header=True, sep = '\t')

### mBERT + mBERT-Galén

In [60]:
arr_model_name = ['mbert', 'mbert_galen']
ens_name = '_'.join(arr_model_name)
ner_model_name = 'mbert'

In [61]:
df_ens_ann, res_metrics = ens_performance(
    ens_name=ens_name, 
    ner_model_name=ner_model_name, 
    arr_model_name=arr_model_name
)
print(res_metrics)

Nº executions: 10
(0.8463, 0.8412, 0.8437)


In [62]:
dic_ens_res[ens_name] = res_metrics

In [61]:
df_ens_ann.to_csv(RES_DIR_ENS + "df_test_preds_c_hier_task_cls_" + '_'.join(arr_model_name) + \
                  ".csv", index=False, header=True, sep = '\t')

### XLM-R + XLM-R-Galén

In [63]:
arr_model_name = ['xlmr', 'xlmr_galen']
ens_name = '_'.join(arr_model_name)
ner_model_name = 'xlmr'

In [64]:
df_ens_ann, res_metrics = ens_performance(
    ens_name=ens_name, 
    ner_model_name=ner_model_name, 
    arr_model_name=arr_model_name
)
print(res_metrics)

Nº executions: 10
(0.8446, 0.8409, 0.8428)


In [65]:
dic_ens_res[ens_name] = res_metrics

In [64]:
df_ens_ann.to_csv(RES_DIR_ENS + "df_test_preds_c_hier_task_cls_" + '_'.join(arr_model_name) + \
                  ".csv", index=False, header=True, sep = '\t')

In [68]:
MULTI_ENS_PREF = "multi_model_"

In [69]:
def add_empty_ner_preds(ens_name, ner_model_name, arr_model_name, 
                        res_dir=RES_DIR_ENS, empty_value=0, 
                        prefix_name=MULTI_ENS_PREF):
    """
    Considering a set of NER predictions as reference, this procedure inserts empty code-pre and code-suf
    samples in the case a certain model does not predict a reference NER sample.
    """
    # Load reference DF of NER predictions
    df_ref_ens_ner = pd.read_csv(res_dir + "df_test_preds_ens_ner_" + ens_name + "_c_hier_task_cls_" + \
                                 ner_model_name + "_ann.csv", header=0, sep='\t')
    assert ~df_ref_ens_ner[['clinical_case', 'start', 'end']].duplicated().any()
    for model_name in arr_model_name:
        # Load DF of NER predictions from the current model
        df_ens_ner = pd.read_csv(res_dir + "df_test_preds_ens_ner_" + ens_name + "_c_hier_task_cls_" + \
                                 model_name + "_ann.csv", header=0, sep='\t')
        assert df_ens_ner.shape[0] <= df_ref_ens_ner.shape[0]
        arr_i_insert = []
        if df_ens_ner.shape[0] < df_ref_ens_ner.shape[0]:
            # Check the indices of the absent samples, i.e. reference NER samples that are not predicted by the current model
            absent_samples = set(df_ref_ens_ner.apply(lambda x: str(x['clinical_case']) + ' ' + str(x['start']) + ' ' + str(x['end']), 
                                                      axis=1)) - \
                             set(df_ens_ner.apply(lambda x: str(x['clinical_case']) + ' ' + str(x['start']) + ' ' + str(x['end']), 
                                                  axis=1))
            assert len(absent_samples) == df_ref_ens_ner.shape[0] - df_ens_ner.shape[0]
            arr_i_absent = []
            for s in absent_samples:
                s = s.split(' ')
                s_index = df_ref_ens_ner[(df_ref_ens_ner['clinical_case'] == s[0]) & (df_ref_ens_ner['start'] == int(s[1])) & \
                                         (df_ref_ens_ner['end'] == int(s[2]))].index[0]
                arr_i_absent.append(np.where(df_ref_ens_ner.index.values == s_index)[0][0])
            assert len(arr_i_absent) > 0
            # Sort indices of absent samples
            arr_i_absent = np.sort(arr_i_absent)
            # Convert indices of absent samples to insertion indices
            arr_i_insert.append(arr_i_absent[0])
            for i in range(1, len(arr_i_absent)):
                arr_i_insert.append(arr_i_absent[i] - i) # needs to be sanity checked
        ## Insert empty values for the absent samples (if any)
        for i_exec in arr_exec:
            # Load predictions of both code prefix and suffix
            preds_code_pre = np.load(file=res_dir + "test_preds_code_pre_ens_ner_" + ens_name + \
                                     "_c_hier_task_cls_" + model_name + "_" + str(i_exec) + ".npy")
            preds_code_suf = np.load(file=res_dir + "test_preds_code_suf_ens_ner_" + ens_name + \
                                     "_c_hier_task_cls_" + model_name + "_" + str(i_exec) + ".npy")
            preds_code_pre = np.insert(arr=preds_code_pre, obj=arr_i_insert, values=empty_value, axis=0)
            preds_code_suf = np.insert(arr=preds_code_suf, obj=arr_i_insert, values=empty_value, axis=0)
            # Save predictions after insertion
            np.save(file=res_dir + prefix_name + "test_preds_code_pre_ens_ner_" + ens_name + \
                    "_c_hier_task_cls_" + model_name + "_" + str(i_exec) + ".npy",
                    arr=preds_code_pre)
            np.save(file=res_dir + prefix_name + "test_preds_code_suf_ens_ner_" + ens_name + \
                    "_c_hier_task_cls_" + model_name + "_" + str(i_exec) + ".npy",
                    arr=preds_code_suf)

### BETO + mBERT + XLM-R

In [70]:
ens_name = "beto_mbert_xlmr"
arr_model_name = ['beto', 'mbert', 'xlmr']

In [71]:
df_ens_iob = pd.read_csv(RES_DIR_ENS + "df_test_preds_c_hier_task_iob_" + ens_name + ".csv", header=0, sep='\t')

In [72]:
df_ens_iob.shape

(3607, 4)

In [73]:
for model_name in arr_model_name:
    df_ens_ner = pd.read_csv(RES_DIR_ENS + "df_test_preds_ens_ner_" + ens_name + "_c_hier_task_cls_" + \
                             model_name + "_ann.csv", header=0, sep='\t')
    print(model_name, "NER predictions:", df_ens_ner.shape[0])

beto NER predictions: 3607
mbert NER predictions: 3607
xlmr NER predictions: 3606


Both BETO and mBERT models could be empoyed as NER-reference, since both make prediction for all NER samples.

In [74]:
ner_model_name = "beto"

In [75]:
add_empty_ner_preds(
    ens_name=ens_name, 
    ner_model_name=ner_model_name, 
    arr_model_name=arr_model_name, 
    empty_value=0
)

In [76]:
df_ens_ann, res_metrics = ens_performance(
    ens_name=ens_name, 
    ner_model_name=ner_model_name, 
    arr_model_name=arr_model_name,
    prefix_name=MULTI_ENS_PREF
)
print(res_metrics)

Nº executions: 15
(0.8453, 0.8393, 0.8423)


In [77]:
dic_ens_res[ens_name] = res_metrics

In [75]:
df_ens_ann.to_csv(RES_DIR_ENS + "df_test_preds_c_hier_task_cls_" + '_'.join(arr_model_name) + \
                  ".csv", index=False, header=True, sep = '\t')

### BETO-Galén + mBERT-Galén + XLM-R-Galén

In [78]:
ens_name = "beto_galen_mbert_galen_xlmr_galen"
arr_model_name = ['beto_galen', 'mbert_galen', 'xlmr_galen']

In [79]:
df_ens_iob = pd.read_csv(RES_DIR_ENS + "df_test_preds_c_hier_task_iob_" + ens_name + ".csv", header=0, sep='\t')

In [80]:
df_ens_iob.shape

(3611, 4)

In [81]:
for model_name in arr_model_name:
    df_ens_ner = pd.read_csv(RES_DIR_ENS + "df_test_preds_ens_ner_" + ens_name + "_c_hier_task_cls_" + \
                             model_name + "_ann.csv", header=0, sep='\t')
    print(model_name, "NER predictions:", df_ens_ner.shape[0])

beto_galen NER predictions: 3610
mbert_galen NER predictions: 3611
xlmr_galen NER predictions: 3610


Only mBERT-Galén models could be empoyed as NER-reference, since it makes prediction for all NER samples.

In [82]:
ner_model_name = "mbert_galen"

In [83]:
add_empty_ner_preds(
    ens_name=ens_name, 
    ner_model_name=ner_model_name, 
    arr_model_name=arr_model_name, 
    empty_value=0
)

In [84]:
df_ens_ann, res_metrics = ens_performance(
    ens_name=ens_name, 
    ner_model_name=ner_model_name, 
    arr_model_name=arr_model_name,
    prefix_name=MULTI_ENS_PREF
)
print(res_metrics)

Nº executions: 15
(0.8518, 0.8467, 0.8493)


In [85]:
dic_ens_res[ens_name] = res_metrics

In [83]:
df_ens_ann.to_csv(RES_DIR_ENS + "df_test_preds_c_hier_task_cls_" + '_'.join(arr_model_name) + \
                  ".csv", index=False, header=True, sep = '\t')

In [86]:
df = pd.DataFrame(dic_ens_res, index=["P", "R", "F1"]).transpose()

In [87]:
df

,P,R,F1
beto,0.8358,0.8282,0.8320
beto_galen,0.8384,0.8337,0.8360
mbert,0.8396,0.8354,0.8375
mbert_galen,0.8433,0.8398,0.8415
xlmr,0.8348,0.8332,0.8340
xlmr_galen,0.8426,0.8382,0.8403
beto_beto_galen,0.8453,0.8346,0.8399
mbert_mbert_galen,0.8463,0.8412,0.8437
xlmr_xlmr_galen,0.8446,0.8409,0.8428
beto_mbert_xlmr,0.8453,0.8393,0.8423


## Multi-task NER

In [90]:
df = format_df_paper(
    model_performance(
        {
            'beto': [1, 2, 3, 4, 5], 
            'beto_galen': [1, 2, 3, 4, 5],
            'mbert': [1, 2, 3, 4, 5], 
            'mbert_galen': [1, 2, 3, 4, 5],
            'xlmr': [1, 2, 3, 4, 5], 
            'xlmr_galen': [1, 2, 3, 4, 5]
        },
        multi_task=True
    )
)

In [91]:
df

,P_avg_std,P_max,R_avg_std,R_max,F1_avg_std,F1_max
beto,.814 ± .013,.834,.809 ± .005,.816,.812 ± .005,.818
beto_galen,.819 ± .005,.826,.82 ± .005,.825,.819 ± .002,.822
mbert,.813 ± .006,.817,.817 ± .004,.821,.815 ± .002,.817
mbert_galen,.825 ± .012,.843,.824 ± .004,.83,.825 ± .005,.832
xlmr,.808 ± .003,.812,.812 ± .006,.821,.81 ± .002,.813
xlmr_galen,.821 ± .008,.835,.821 ± .003,.825,.821 ± .004,.827


Save the (F1) performance of all executions of all models

In [8]:
def multi_model_f1_values(dict_names_execs, 
                    df_gs=df_test_gs, path_gs=test_gs_path):
    """
    Generate a vector containing the F1 performance of all executions of all models, in the given order.
    
    dict_names_execs: each key is a string with the model name, and 
                      each value is a list with the random execs of the corresponding model.
    """
    arr_values = []
    for model_name in dict_names_execs:
        for i_exec in dict_names_execs[model_name]:
            df_test_preds = pd.read_csv(RES_DIR + "df_test_preds_multi_task_ner_" + str(i_exec) + "_c_hier_task_cls_" + \
                    str(model_name) + "_" + str(i_exec) + ".csv", header=0, sep='\t')
            _, _, f1 = calculate_ner_metrics(
                gs=df_gs, 
                pred=format_ner_pred_df(
                    gs_path=path_gs, 
                    df_preds=df_test_preds, 
                    subtask='norm'
                ),
                subtask='norm'
            )
            arr_values.append(f1)
    return arr_values

In [ ]:
# NORM

In [9]:
arr_val = multi_model_f1_values(
    {
        'beto': [1, 2, 3, 4, 5], 
        'beto_galen': [1, 2, 3, 4, 5],
        'mbert': [1, 2, 3, 4, 5], 
        'mbert_galen': [1, 2, 3, 4, 5],
        'xlmr': [1, 2, 3, 4, 5], 
        'xlmr_galen': [1, 2, 3, 4, 5]
    }
)

In [10]:
pd.DataFrame(arr_val).to_csv(RES_DIR + "norm_f1_exec_c_multi_task_ner_hier_task.csv", index=False, header=False, sep = '\t')